In [29]:
from typing import Dict, Text
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import numpy as np

#### Read the data

In [18]:
# ratings data
ratings = tfds.load('movielens/100k-ratings', split='train')

# Features of all the available movies
movies = tfds. load('movielens/100k-movies', split='train')

# select the basic features
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})

movies = movies.map(lambda x: x["movie_title"])

In [19]:
# Build vocabularies to convert user ids and movie titles into integer indices for embedding layers

user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

#### Define a model

We can define a TFRS model by inheriting from 'tfrs.Model' and implementing the 'compute_loss' method

In [26]:
class MovieLensModel(tfrs.Model):
    # we derive from a custom base class to help reduce boilerplate. Under the hood,
    # these are still plain Keras Models.

    def __init__(
        self,
        user_model: tf.keras.Model,
        movie_model: tf.keras.Model,
        task: tfrs.tasks.Retrieval
    ):
        super().__init__()

        # set up user and movie representations
        self.user_model = user_model
        self.movie_model = movie_model

        # set up a retrieval task
        self.task = task

    def compute_loss(
        self,
        features: Dict[Text, tf.Tensor],
        training=False
    ) -> tf.Tensor:
        # define how the loss is computed

        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        return self.task(user_embeddings, movie_embeddings)

define the two models and the retrieval task

In [27]:
# define user and movie models
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])

# Define your objectives
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
))

#### Fit and evaluate it

create the model, train it, and generate predictions

In [32]:
# create a retrieval model
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# train for 3 epochs
model.fit(ratings.batch(4096), epochs=3)

# use brute-force search to set up retrieval using the trained representations
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title)))
)

# Get some recommendations
_, titles = index(np.array(["123"]))
print(f"Top 5 recommendations for user 123: {titles[0, :5]}")

Epoch 1/3
25/25 [==============================] - 4s 140ms/step - factorized_top_k/top_1_categorical_accuracy: 4.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0055 - factorized_top_k/top_10_categorical_accuracy: 0.0128 - factorized_top_k/top_50_categorical_accuracy: 0.0891 - factorized_top_k/top_100_categorical_accuracy: 0.1853 - loss: 33208.2981 - regularization_loss: 0.0000e+00 - total_loss: 33208.2981
Epoch 2/3
25/25 [==============================] - 4s 140ms/step - factorized_top_k/top_1_categorical_accuracy: 5.0000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0081 - factorized_top_k/top_10_categorical_accuracy: 0.0211 - factorized_top_k/top_50_categorical_accuracy: 0.1452 - factorized_top_k/top_100_categorical_accuracy: 0.2738 - loss: 30502.9075 - regularization_loss: 0.0000e+00 - total_loss: 30502.9075
Epoch 3/3
25/25 [==============================] - 4s 139ms/step - factorized_top_k/top_1_categorical_accuracy: 4.3000e-04 - factorized_top_k/top_5_categorica